# Azure AI Agent Service

* Azure SDK: [azure.ai.projects](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#agents-preview)
* Azure Sample: [Enterprise Demo](https://github.com/Azure-Samples/azure-ai-agent-service-enterprise-demo/tree/main)

In [ ]:
# Import Dependencies
import os
import datetime
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

## Reusable Functions

In [ ]:
# Function to Connect Client
def connect_client():
  try:
    return AIProjectClient.from_connection_string(
      credential=DefaultAzureCredential(),
      conn_str=os.environ["PROJECT_CONNECTION_STRING"]
    )
  except Exception as e:
    print(e)
    return None

# Function to Create Agent
def create_agent(
  agent_name = f"Agent {datetime.datetime.now().strftime("%Y%m%d%H%M")}",
  agent_instructions = "You are a helpful assistant.",
  agent_model = os.environ["MODEL_DEPLOYMENT_NAME"],
  agent_toolset = None,
):
  try:
    project_client = connect_client()
    return project_client.agents.create_agent(
      name = agent_name,
      instructions = agent_instructions,
      model = agent_model,
      toolset = agent_toolset,
    )
  except Exception as e:
    print(e)
    return None

# Function to List Agents
def list_agents():
  try:
    project_client = connect_client()
    return project_client.agents.list_agents()
  except Exception as e:
    print(e)
    return None

# Function to Ask Agent
def ask_agent(agent_id, prompt):
  # Default AI Response
  ai_response = "I don't know what to say."

  # Connect Project Client
  project_client = connect_client()

  # Validate project client
  if project_client:
    # Create thread for communication
    try:
      thread = project_client.agents.create_thread()
      # print(f"Created thread, ID: {thread.id}")
    except Exception as e:
      print(e)
      return None
    
    # Validate thread
    if thread is not None and thread.id is not None:
      # Create message to thread
      try:
        message = project_client.agents.create_message(
          thread_id=thread.id,
          role="user",
          content=prompt,
        )
        # print(f"Created message, ID: {message.id}")
      except Exception as e:
        print(e)
        return None
      
      # Validate message
      if message is not None and message.id is not None:
        # Create and process agent run in thread
        try:
          run = project_client.agents.create_and_process_run(
            thread_id=thread.id,
            agent_id=agent_id,
          )
          # print(f"Run finished with status: {run.status}")
          if run.status == "failed":
            print(f"Run failed: {run.last_error}")
            return None
        except Exception as e:
          print(e)
          return None

        # Get all messages in thread
        try:
          messages = project_client.agents.list_messages(thread_id=thread.id)
        except Exception as e:
          print(e)
          return None

        # Validate messages
        if messages is not None or messages.data is not None:
          # Get latest message
          try:
              latest_content = messages.data[0].content[0]
          except (IndexError, AttributeError, TypeError):
              latest_content = None

          # Handle text messages
          if (
              latest_content 
              and latest_content.type == "text"
              and latest_content.text
              and latest_content.text.value
          ):
              ai_response = latest_content.text.value

  return ai_response

## Create Agents

In [ ]:
# Create Agent
new_agent = create_agent(
  agent_name = f"Test Agent {datetime.datetime.now().strftime("%Y%m%d%H%M")}",
  agent_instructions = "You are a helpful assistant.",
  agent_model = "gpt-4o",
  agent_toolset = None,
)

# List Agents

In [ ]:
# List All Agents
agents = list_agents()
for agent in agents.data:
  print(f"# {agent.name} ({agent.id})")
  print(f"Instructions: {agent.instructions}")
  print(f"Tools: {agent.tools}\n")

## Ask Agent

In [ ]:
# Ask Agent
agent_id = agents.data[0].id
agent_response_content = ask_agent(
  agent_id,
  "Hello, tell me a unique and funny joke."
)
print(agent_response_content)

## Agents with Built-in Tools

In [ ]:
# Agent with Tools
from azure.ai.projects.models import ToolSet
from azure.ai.projects.models import CodeInterpreterTool

# Create Agent with Tools
try:
  code_interpreter = CodeInterpreterTool()
  toolset = ToolSet()
  toolset.add(code_interpreter)
except Exception as e:
  print(e)

# Create Agent with Code Toolset
agent_coder = create_agent(
  agent_name="Coder Agent",
  agent_instructions="You are a helpful coding agent who knows how to write code.",
  agent_toolset=toolset,
)

In [ ]:
# List All Agents
agents = list_agents()
for agent in agents.data:
  print(f"# {agent.name} ({agent.id}) with {agent.tools}")

In [ ]:
# Prompt to test math computation
prompt = "What is the square root of 987654321 rounded to 5 decimal places?"
agent_with_tools = "" # agents.data[0].id
agent_without_tools = "" # agents.data[1].id

# Ask Agent with Tools
agent_response_content = ask_agent(agent_with_tools, prompt)
print(f"### Agent With Tools ###\n{agent_response_content}")

# Ask Agent without Tools
agent_response_content = ask_agent(agent_without_tools, prompt)
print(f"\n### Agent Without Tools ###\n{agent_response_content}")

In [ ]:
# Prompt to test data analysis
prompt = "Can you generate and display a table of 10 random numbers between 1 and 100 and compute their mean and standard deviation?"
agent_with_tools = "" # agents.data[0].id
agent_without_tools = "" # agents.data[1].id

# Ask Agent with Tools
agent_response_content = ask_agent(agent_with_tools, prompt)
print(f"### Agent With Tools ###\n{agent_response_content}")

# Ask Agent without Tools
agent_response_content = ask_agent(agent_without_tools, prompt)
print(f"\n### Agent Without Tools ###\n{agent_response_content}")

## Agent with Custom Tools

In [ ]:
# Agent with Custom Tools
from azure.ai.projects.models import FunctionTool, ToolSet
from azure.ai.projects.models import CodeInterpreterTool
from user_functions import user_functions # user functions which can be found in a user_functions.py file.

# Create Agent with Tools
try:
  functions = FunctionTool(user_functions)
  toolset = ToolSet()
  toolset.add(functions)
  
except Exception as e:
  print(e)

agent_weather = create_agent(
  agent_name="Weather Agent",
  agent_instructions="You are a helpful weather assistant who knows how to write code.",
  agent_toolset=toolset,
)

In [ ]:
# List All Agents
agents = list_agents()
for agent in agents.data:
  print(f"# {agent.name} ({agent.id}) with {agent.tools}")

In [ ]:
# Ask Agent
agent_response_content = ask_agent(
  "",
  "What is the weather in New York City today?"
)
print(agent_response_content)